# Custom dataset

Test class to create a custom MNIST dataset for  generating number from 0 to 999 in a single image

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
from torch.utils.data import Dataset
import random
from PIL import Image

# Train dataset

In [ ]:
mnist_mean = 0.1307
mnist_std = 0.3081

batch_size_train = 64
batch_size_test = 1000

transform_train = transforms.Compose(
    [transforms.ConvertImageDtype(torch.float32),
    transforms.Normalize(mnist_mean, mnist_std)]
)

to_pil = transforms.ToPILImage()

def denormalize_train(img):
    img = img*mnist_std
    return img + mnist_mean

In [ ]:
class DynamicWMNIST(torchvision.datasets.MNIST):
    def __init__(self, root="./data", train=True, min_digits=1, max_digits=3, dataset_size=500000, transform=None, download=True):
        super().__init__(root=root, train=train, transform=transform, download=download)

        self.min_digits = min_digits
        self.max_digits = max_digits
        self.dataset_size = dataset_size
        self.transform = transform

        self.label_to_indices = {
            i: torch.where(self.targets == i)[0] for i in range(10)
        }

    def __len__(self):
        return self.dataset_size

    def __getitem__(self, idx):
        n_digits = random.randint(self.min_digits,self.max_digits)
        number = random.randint(0 if n_digits == 1 else 10**(n_digits-1), 10**n_digits - 1)
        digits = list(str(number))

        digit_images = []
        for d in digits:
            label = int(d)
            indices = self.label_to_indices[label]
            chosen_idx = indices[torch.randint(len(indices), (1,)).item()]
            img = self.data[chosen_idx]
            img = img.unsqueeze(0)
            digit_images.append(img)
        concat_img = torch.cat(digit_images, dim=2)

        if self.transform:
            concat_img = self.transform(concat_img)

        return concat_img, number


In [ ]:
class DynamicDMNIST(torchvision.datasets.MNIST):
    def __init__(self, root="./data", train=True, min_digits=1, max_digits=3, dataset_size=500000, transform=None, download=True):
        super().__init__(root=root, train=train, transform=transform, download=download)

        self.min_digits = min_digits
        self.max_digits = max_digits
        self.dataset_size = dataset_size
        self.transform = transform

        self.label_to_indices = {
            i: torch.where(self.targets == i)[0] for i in range(10)
        }

    def __len__(self):
        return self.dataset_size

    def __getitem__(self, idx):
        n_digits = random.randint(self.min_digits,self.max_digits)
        number = random.randint(0 if n_digits == 1 else 10**(n_digits-1), 10**n_digits - 1)
        digits = list(str(number))

        digit_images = []
        for d in digits:
            label = int(d)
            indices = self.label_to_indices[label]
            chosen_idx = indices[torch.randint(len(indices), (1,)).item()]
            img = self.data[chosen_idx]
            img = img.unsqueeze(0)
            digit_images.append(img)
        for i in range(n_digits, self.max_digits):
            img = torch.zeros((28, 28))
            img = img.unsqueeze(0)
            digit_images.append(img)
        concat_img = torch.cat(digit_images, dim=0)

        if self.transform:
            concat_img = self.transform(concat_img)

        return concat_img, number


In [ ]:
def my_collate(batch):
    imgs, labels = zip(*batch)
    return list(imgs), list(labels)

In [ ]:
train_w_dataset = DynamicWMNIST(transform=transform_train)
train_w_dataloader = torch.utils.data.DataLoader(
    train_w_dataset,
    batch_size=batch_size_train,
    shuffle=True,
    collate_fn=my_collate
)

In [ ]:
train_d_dataset = DynamicDMNIST(transform=transform_train)
train_d_dataloader = torch.utils.data.DataLoader(
    train_d_dataset,
    batch_size=batch_size_train,
    shuffle=True,
    collate_fn=my_collate
)

# Testset
adesso iniziero a creare il testset

In [ ]:
import os

# Specify the directory name
directory_name = "data/TestWMNIST"

# Create the directory
try:
    os.mkdir(directory_name)
    print(f"Directory '{directory_name}' created successfully.")
except FileExistsError:
    print(f"Directory '{directory_name}' already exists.")
except PermissionError:
    print(f"Permission denied: Unable to create '{directory_name}'.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
for i in range(0,1000):
    # Specify the directory name
    directory_name = "data/TestWMNIST/" + str(i)

    # Create the directory
    try:
        os.mkdir(directory_name)
        print(f"Directory '{directory_name}' created successfully.")
    except FileExistsError:
        print(f"Directory '{directory_name}' already exists.")
    except PermissionError:
        print(f"Permission denied: Unable to create '{directory_name}'.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
from PIL import Image

ds = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    transform=None,
    download=True
)

label_indices = { i:torch.where(ds.targets == i)[0] for i in range(10) }
save_dir = "data/TestWMNIST"

for i in range(0, 10):
    for j in range(0, 1500):
        indices = label_indices[i]
        chosen_idx = indices[torch.randint(len(indices), (1,)).item()]
        img = to_pil(ds.data[chosen_idx])
        img.save(os.path.join(save_dir + "/" + str(i), str(j) + ".png"))

for i in range(10, 100):
    for j in range(0, 167):
        digits = list(str(i))
        digit_images = []
        for d in digits:
            indices = label_indices[int(d)]
            chosen_idx = indices[torch.randint(len(indices), (1,)).item()]
            img = ds.data[chosen_idx]
            digit_images.append(img)
            concat_img = torch.cat(digit_images, dim=1)
            img = to_pil(concat_img)
        img.save(os.path.join(save_dir + "/" + str(i), str(j) + ".png"))

for i in range(100, 1000):
    for j in range(0, 17):
        digits = list(str(i))
        digit_images = []
        for d in digits:
            indices = label_indices[int(d)]
            chosen_idx = indices[torch.randint(len(indices), (1,)).item()]
            img = ds.data[chosen_idx]
            digit_images.append(img)
            concat_img = torch.cat(digit_images, dim=1)
            img = to_pil(concat_img)
        img.save(os.path.join(save_dir + "/" + str(i), str(j) + ".png"))

In [ ]:
import os

# Specify the directory name
directory_name = "data/TestDMNIST"

# Create the directory
try:
    os.mkdir(directory_name)
    print(f"Directory '{directory_name}' created successfully.")
except FileExistsError:
    print(f"Directory '{directory_name}' already exists.")
except PermissionError:
    print(f"Permission denied: Unable to create '{directory_name}'.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
for i in range(0,1000):
    # Specify the directory name
    directory_name = "data/TestDMNIST/" + str(i)

    # Create the directory
    try:
        os.mkdir(directory_name)
        print(f"Directory '{directory_name}' created successfully.")
    except FileExistsError:
        print(f"Directory '{directory_name}' already exists.")
    except PermissionError:
        print(f"Permission denied: Unable to create '{directory_name}'.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
from PIL import Image

ds = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    transform=None,
    download=True
)

max_digits = 3

label_indices = { i:torch.where(ds.targets == i)[0] for i in range(10) }
save_dir = "data/TestDMNIST"

def get_number(idx, j):
    digits = list(str(idx))
    digit_images = []
    for d in digits:
        indices = label_indices[int(d)]
        chosen_idx = indices[torch.randint(len(indices), (1,)).item()]
        img = ds.data[chosen_idx]
        img = img.unsqueeze(0)
        digit_images.append(img)
    for d in range(len(digits), max_digits):
        img = torch.zeros((28, 28))
        img = img.unsqueeze(0)
        digit_images.append(img)
    concat_img = torch.cat(digit_images, dim=0)
    img = to_pil(concat_img)
    img.save(os.path.join(save_dir + "/" + str(i), str(j) + ".png"))
    

for i in range(0, 1000):
    if i < 10:
        for j in range(0, 1500):
            get_number(i, j)
    elif i < 100:
        for j in range(0, 167):
            get_number(i, j)
    else:
        for j in range(0, 17):
            get_number(i, j)

In [ ]:
import os
import numpy as np
from PIL import Image

# percorso al tuo dataset
dataset_path = "./data/TestWMNIST"

to_tensor = torchvision.transforms.ToTensor()

pixels = []
i = 0
# scansiona ricorsivamente cartelle e sottocartelle
for root, _, files in os.walk(dataset_path):
    for file in files:
        print(i)
        i += 1
        if file.lower().endswith((".png")):
            img_path = os.path.join(root, file)
            img = Image.open(img_path).convert("L")  # o "L" per grayscale
            img = to_tensor(img)
            arr = np.array(img, dtype=np.float32)
            pixels.append(arr.flatten())

# unisce tutte le immagini in un unico array
all_pixels = np.concatenate(pixels)

# calcolo media e varianza
mean = np.mean(all_pixels, axis=0)
std = np.var(all_pixels, axis=0)

print("Media:", mean)
print("Varianza:", std)


In [ ]:
transformW = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean, std)
])

def denormalizeW(img):
    img = img * std
    return img + mean

In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

class testMNISTDataset(Dataset):
    def __init__(self, root, transform=None):
        self.transform = transform
        self.samples = []
        self.class_to_idx = {}
        
        # leggo le cartelle e ordino numericamente
        classes = sorted([d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))], key=int)
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        
        # creo lista (path_file, label)
        for cls_name in classes:
            cls_idx = self.class_to_idx[cls_name]
            folder_path = os.path.join(root, cls_name)
            files = sorted(os.listdir(folder_path), key = lambda x: int(x.removesuffix('.png')))
            for fname in files:
                self.samples.append((os.path.join(folder_path, fname), cls_idx))

    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path)
        if self.transform:
            img = self.transform(img)
        return img, label

In [ ]:
testWMNIST = testMNISTDataset('./data/testWMNIST/', transformW)

In [ ]:
from torch.utils.data import DataLoader

test_dataloaderW = DataLoader(testWMNIST, # dataset to iterate
                              batch_size=10, # how many images to load every iteration
                              shuffle=False) # sample data

In [ ]:
batch = next(iter(test_dataloaderW))
data, label = batch

print(label[0])
plt.imshow(to_pil(denormalizeW(data[0])))

In [ ]:
import os
import numpy as np
from PIL import Image

# percorso al tuo dataset
dataset_path = "./data/TestDMNIST"

to_tensor = torchvision.transforms.ToTensor()

pixels = []
i = 0
# scansiona ricorsivamente cartelle e sottocartelle
for root, _, files in os.walk(dataset_path):
    for file in files:
        print(i)
        i += 1
        if file.lower().endswith((".png")):
            img_path = os.path.join(root, file)
            img = Image.open(img_path)
            img = to_tensor(img)
            arr = np.array(img, dtype=np.float32)
            pixels.append(arr.flatten())

# unisce tutte le immagini in un unico array
all_pixels = np.concatenate(pixels)

# calcolo media e varianza
meanD = np.mean(all_pixels, axis=0)
stdD = np.var(all_pixels, axis=0)

print("Media:", meanD)
print("Varianza:", stdD)


In [ ]:
transformD = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(meanD, stdD)
])

def denormalizeD(img):
    img = img * stdD
    return img + meanD

In [ ]:
testDMNIST = testMNISTDataset('./data/TestDMNIST', transformD)

In [ ]:
from torch.utils.data import DataLoader

train_dataloaderD = DataLoader(testDMNIST, # dataset to iterate
                              batch_size=10, # how many images to load every iteration
                              shuffle=False) # sample data

In [ ]:
batch = next(iter(train_dataloaderD))
data, label = batch
print(label)
figure = plt.figure(figsize=(10, 10))
for i in range(0, 10):
    figure.add_subplot(10, 10, i+1)
    plt.axis("off")
    plt.imshow(to_pil(denormalizeD(data[i])))

In [ ]:
batch = testDMNIST[15335]
print(testDMNIST.__len__())
data, label = batch
print(label)
plt.imshow(to_pil(denormalizeD(data[1])))